In [46]:
from pandas import DataFrame
from ucimlrepo import fetch_ucirepo
from typing import NamedTuple, List, Dict, Any
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from pprint import pprint


# Type definitions from:
# https://github.com/uci-ml-repo/ucimlrepo?tab=readme-ov-file#parameters
class UciMlData(NamedTuple):
    ids: DataFrame
    features: DataFrame
    targets: DataFrame
    original: DataFrame
    headers: List[str]


class UciMlDataset(NamedTuple):
    data: UciMlData
    metadata: Dict[str, Any]
    variables: DataFrame

In [9]:
iris: UciMlDataset = fetch_ucirepo(id=53)

In [6]:
X = iris.data.features
y = iris.data.targets

display(X.head())
display(y.head())

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


,class
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


# KNN on IRIS (without crossvalidation)

In [50]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline


def perform_cross_validation(
    pipeline: Pipeline,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    folds: int = 5,
):
    """
    Perform cross-validation and return the mean scores for specified metrics.

    Parameters:
    - pipeline: The machine learning pipeline that includes preprocessing and a classifier.
    - X_train: Training feature set as a pandas DataFrame.
    - y_train: Training target as a pandas Series.
    - folds: Number of cross-validation folds.

    Returns:
    - A dictionary containing mean scores for accuracy, precision, recall, and F1 score.
    """
    scoring = {
        "accuracy": "accuracy",
        "precision": "precision_macro",
        "recall": "recall_macro",
        "f1": "f1_macro",
    }

    cv_results = cross_validate(pipeline, X_train, y_train, scoring=scoring, cv=folds)

    mean_scores = {
        metric: np.mean(scores)
        for metric, scores in cv_results.items()
        if "test_" in metric
    }

    return mean_scores


def run_knn(
    X: DataFrame,
    y: DataFrame,
    K: int = 5,
):
    y = y.squeeze()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    pipeline = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("knn", KNeighborsClassifier(n_neighbors=K, metric="euclidean")),
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    report = classification_report(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)

    return report, conf_mat


def run_knn_cross_validation(
    X: DataFrame,
    y: DataFrame,
    K: int = 5,
):
    y = y.squeeze()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    pipeline = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("knn", KNeighborsClassifier(n_neighbors=K, metric="euclidean")),
        ]
    )

    results = perform_cross_validation(pipeline, X_train, y_train)
    return results

In [38]:
report, conf_mat = run_knn(X, y)
print(report)
print(conf_mat)

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


In [57]:
def test_cross_validation(X: pd.DataFrame, Y: pd.DataFrame):
    fold_nums = [2, 5, 10]

    for folds in fold_nums:
        res = run_knn_cross_validation(X, y, folds)
        print(f"Results for {folds}-fold cross-validation:")
        display(pd.DataFrame(res, index=[0]))

In [58]:
test_cross_validation(X, y)

Results for 2-fold cross-validation:


,test_accuracy,test_precision,test_recall,test_f1
0,0.933333,0.942328,0.932143,0.932403


Results for 5-fold cross-validation:


,test_accuracy,test_precision,test_recall,test_f1
0,0.933333,0.941481,0.932143,0.932203


Results for 10-fold cross-validation:


,test_accuracy,test_precision,test_recall,test_f1
0,0.941667,0.948254,0.940476,0.940968


## Crossvalidation results for IRIS dataset

With crossvalidation f1 scores where worse than without crossvalidation.

### Fold sizes comparison

The best result (f1 score) was obtained with 10 folds. 
